In [16]:
import os
import glob
import pandas as pd

RUTA_LOGS = "."
RUTA_TEST = "."   # donde están los *_test.csv

archivos_xlsx = [
    f for f in glob.glob(os.path.join(RUTA_LOGS, "*.xlsx"))
    if not os.path.basename(f).startswith("~$")
]

archivos_test = glob.glob(os.path.join(RUTA_TEST, "*_test.csv"))

# ---- cargar distribuciones de test ----
dist_test = {}
for ruta in archivos_test:
    nombre = os.path.basename(ruta)
    dataset = nombre.split("_tdataset")[0]
    df_test = pd.read_csv(ruta)
    col_clase = df_test.columns[-1]
    dist_test[dataset] = df_test[col_clase].value_counts()

# ---- análisis principal ----
for ruta in archivos_xlsx:
    nombre_archivo = os.path.basename(ruta)
    dataset = (
        nombre_archivo
        .replace("log_pcsmote_x_muestra_", "")
        .replace(".xlsx", "")
    )

    df = pd.read_excel(ruta, engine="openpyxl")

    columnas = {
        "configuracion",
        "clase_objetivo",
        "synthetics_from_this_seed",
        "es_semilla_valida"
    }
    if not columnas <= set(df.columns):
        raise ValueError(f"Columnas faltantes en {nombre_archivo}")

    print("\n" + "=" * 100)
    print(f"Dataset: {dataset}")
    print("=" * 100)

    for configuracion, bloque in df.groupby("configuracion"):

        print(f"\n{configuracion}")
        print("-" * len(configuracion))

        resumen = (
            bloque
            .groupby("clase_objetivo")
            .agg(
                SG=("synthetics_from_this_seed", "sum"),
                SV=("es_semilla_valida", "sum")
            )
            .sort_index()
        )

        # imprimir por clase
        for clase, fila in resumen.iterrows():
            sg = int(fila["SG"])
            sv = int(fila["SV"])
            ratio = sg / sv if sv > 0 else 0
            print(f"{clase:<6} SV={sv:<3} SG={sg:<4} SG/SV={ratio:.2f}")

        # métricas globales
        total_sg = resumen["SG"].sum()
        cobertura = (resumen["SG"] > 0).sum()
        total_clases = len(resumen)

        print(f"\n  ▶ Total sintéticos (SG): {int(total_sg)}")
        print(f"  ▶ Cobertura de clases  : {cobertura}/{total_clases}")

        if total_sg == 0:
            print("  ⚠ Configuración inválida (SG = 0)")
            continue

        # ---- desalineación con test ----
        if dataset in dist_test:
            print("  ▶ Desalineación SG vs TEST:")
            for clase in resumen.index:
                sg = resumen.loc[clase, "SG"]
                ft = dist_test[dataset].get(clase, 0)
                delta = sg - ft
                print(f"     {clase:<6} SG={sg:<4} TEST={ft:<3} Δ={delta}")



Dataset: ecoli

PRD70_PR30_CPprop_UD060_UR040_Upp6000_I0
----------------------------------------
im     SV=0   SG=0    SG/SV=0.00
imU    SV=0   SG=0    SG/SV=0.00
om     SV=0   SG=0    SG/SV=0.00
pp     SV=0   SG=0    SG/SV=0.00

  ▶ Total sintéticos (SG): 0
  ▶ Cobertura de clases  : 0/4
  ⚠ Configuración inválida (SG = 0)

PRD70_PR30_CPprop_UD060_UR040_Upp6000_I1
----------------------------------------
im     SV=0   SG=0    SG/SV=0.00
imU    SV=0   SG=0    SG/SV=0.00
om     SV=0   SG=0    SG/SV=0.00
pp     SV=0   SG=0    SG/SV=0.00

  ▶ Total sintéticos (SG): 0
  ▶ Cobertura de clases  : 0/4
  ⚠ Configuración inválida (SG = 0)

PRD70_PR30_CPprop_UD060_UR040_Upp6000_I3
----------------------------------------
im     SV=0   SG=0    SG/SV=0.00
imU    SV=0   SG=0    SG/SV=0.00
om     SV=0   SG=0    SG/SV=0.00
pp     SV=0   SG=0    SG/SV=0.00

  ▶ Total sintéticos (SG): 0
  ▶ Cobertura de clases  : 0/4
  ⚠ Configuración inválida (SG = 0)

PRD70_PR40_CPent_UD060_UR045_PE60_I0
----------

In [2]:
import os
import glob
import pandas as pd


# =========================
# UTILIDADES
# =========================
def tiene_columnas(df: pd.DataFrame, columnas: list) -> bool:
    for c in columnas:
        if c not in df.columns:
            return False
    return True


def validar_columnas_obligatorias(df: pd.DataFrame, nombre_archivo: str) -> None:
    columnas_obligatorias = [
        "configuracion",
        "clase_objetivo",
        "synthetics_from_this_seed",
        "es_semilla_valida",
    ]

    faltantes = []
    for c in columnas_obligatorias:
        if c not in df.columns:
            faltantes.append(c)

    if len(faltantes) > 0:
        raise ValueError(f"Columnas faltantes en {nombre_archivo}: {faltantes}")


def convertir_a_float_seguro(serie: pd.Series) -> pd.Series:
    """
    Convierte una serie a float de forma robusta:
    - numéricos -> float
    - strings "7/7" -> 1.0
    - strings con coma decimal -> reemplaza coma por punto
    - cualquier otra cosa no convertible -> NaN
    """
    # Paso 1: forzamos a string para procesar patrones, pero preservamos NaN
    s = serie.copy()

    # Normalizamos a string sin perder NaN
    s = s.astype("object")

    valores_convertidos = []

    for v in s.values:
        if pd.isna(v):
            valores_convertidos.append(float("nan"))
            continue

        # Si ya es numérico
        if isinstance(v, (int, float)):
            valores_convertidos.append(float(v))
            continue

        texto = str(v).strip()

        if texto == "":
            valores_convertidos.append(float("nan"))
            continue

        # Caso "a/b"
        if "/" in texto:
            partes = texto.split("/")
            if len(partes) == 2:
                a_txt = partes[0].strip()
                b_txt = partes[1].strip()
                try:
                    a = float(a_txt.replace(",", "."))
                    b = float(b_txt.replace(",", "."))
                    if b != 0:
                        valores_convertidos.append(a / b)
                    else:
                        valores_convertidos.append(float("nan"))
                except Exception:
                    valores_convertidos.append(float("nan"))
                continue

        # Caso decimal con coma
        texto = texto.replace(",", ".")

        try:
            valores_convertidos.append(float(texto))
        except Exception:
            valores_convertidos.append(float("nan"))

    return pd.Series(valores_convertidos, index=serie.index)


# =========================
# CARGA DISTRIBUCIONES TEST
# =========================
def cargar_distribuciones_test(ruta_test: str) -> dict:
    archivos_test = glob.glob(os.path.join(ruta_test, "*_test.csv"))
    dist_test = {}

    for ruta in archivos_test:
        nombre = os.path.basename(ruta)
        dataset = nombre.split("_tdataset")[0]

        df_test = pd.read_csv(ruta)
        col_clase = df_test.columns[-1]
        dist_test[dataset] = df_test[col_clase].value_counts()

    return dist_test


# =========================
# ANÁLISIS 1: SG/SV POR CLASE
# =========================
def construir_resumen_sg_sv_por_clase(bloque: pd.DataFrame) -> pd.DataFrame:
    resumen = (
        bloque
        .groupby("clase_objetivo")
        .agg(
            SG=("synthetics_from_this_seed", "sum"),
            SV=("es_semilla_valida", "sum"),
        )
        .sort_index()
    )
    return resumen


def imprimir_resumen_sg_sv(resumen: pd.DataFrame) -> int:
    for clase, fila in resumen.iterrows():
        sg = int(fila["SG"])
        sv = int(fila["SV"])
        ratio = sg / sv if sv > 0 else 0
        print(f"{clase:<6} SV={sv:<3} SG={sg:<4} SG/SV={ratio:.2f}")

    total_sg = int(resumen["SG"].sum())
    cobertura = int((resumen["SG"] > 0).sum())
    total_clases = int(len(resumen))

    print(f"\n  ▶ Total sintéticos (SG): {total_sg}")
    print(f"  ▶ Cobertura de clases  : {cobertura}/{total_clases}")

    if total_sg == 0:
        print("  ⚠ Configuración inválida (SG = 0)")

    return total_sg


# =========================
# ANÁLISIS 2: DESALINEACIÓN VS TEST
# =========================
def imprimir_desalineacion_vs_test(dataset: str, resumen: pd.DataFrame, dist_test: dict) -> None:
    if dataset not in dist_test:
        return

    print("  ▶ Desalineación SG vs TEST:")
    for clase in resumen.index:
        sg = int(resumen.loc[clase, "SG"])
        ft = int(dist_test[dataset].get(clase, 0))
        delta = sg - ft
        print(f"     {clase:<6} SG={sg:<4} TEST={ft:<3} Δ={delta}")


# =========================
# ANÁLISIS 3: ENTROPÍA (media/std/CV) EN SV=1
# =========================
def analizar_entropia_en_semillas_validas(bloque: pd.DataFrame) -> None:
    if not tiene_columnas(bloque, ["entropia", "es_semilla_valida"]):
        print("  ▶ Entropía: (omitido, faltan columnas)")
        return

    semillas_validas = bloque[bloque["es_semilla_valida"] == 1]
    if len(semillas_validas) == 0:
        print("  ▶ Entropía: sin semillas válidas")
        return

    ent = convertir_a_float_seguro(semillas_validas["entropia"])
    ent = ent.dropna()

    if len(ent) == 0:
        print("  ▶ Entropía: todos NaN tras conversión")
        return

    media = float(ent.mean())
    std = float(ent.std(ddof=1)) if len(ent) > 1 else 0.0
    cv = (std / media) if media != 0 else 0.0

    print("  ▶ Entropía (solo SV=1):")
    print(f"     n={len(ent)} | media={media:.4f} | std={std:.4f} | CV={cv:.4f}")


# =========================
# ANÁLISIS 4: CORRELACIÓN entropía ↔ SG_por_semilla (Spearman) EN SV=1
# =========================
def analizar_correlacion_entropia_vs_sg(bloque: pd.DataFrame) -> None:
    cols = ["entropia", "synthetics_from_this_seed", "es_semilla_valida"]
    if not tiene_columnas(bloque, cols):
        print("  ▶ Correlación entropía↔SG: (omitido, faltan columnas)")
        return

    semillas_validas = bloque[bloque["es_semilla_valida"] == 1].copy()
    if len(semillas_validas) < 3:
        print("  ▶ Correlación entropía↔SG: datos insuficientes (n<3)")
        return

    x = convertir_a_float_seguro(semillas_validas["entropia"])
    y = convertir_a_float_seguro(semillas_validas["synthetics_from_this_seed"])

    # Alineamos y tiramos NaN
    tmp = pd.DataFrame({"x": x, "y": y}).dropna()
    if len(tmp) < 3:
        print("  ▶ Correlación entropía↔SG: datos insuficientes tras limpieza")
        return

    corr = tmp["x"].corr(tmp["y"], method="spearman")
    if pd.isna(corr):
        print("  ▶ Correlación entropía↔SG: no calculable")
        return

    print("  ▶ Correlación (Spearman) entropía ↔ SG_por_semilla (solo SV=1):")
    print(f"     rho={float(corr):.4f}")


# =========================
# ANÁLISIS 5: EFICIENCIA DE SEMILLAS (SV activas)
# =========================
def analizar_eficiencia_de_semillas(bloque: pd.DataFrame) -> None:
    cols = ["es_semilla_valida", "synthetics_from_this_seed"]
    if not tiene_columnas(bloque, cols):
        print("  ▶ Eficiencia semillas: (omitido, faltan columnas)")
        return

    semillas_validas = bloque[bloque["es_semilla_valida"] == 1].copy()
    if len(semillas_validas) == 0:
        print("  ▶ Eficiencia semillas: sin semillas válidas")
        return

    total_sv = int(len(semillas_validas))

    sg_por_semilla = convertir_a_float_seguro(semillas_validas["synthetics_from_this_seed"])
    semillas_validas = semillas_validas.copy()
    semillas_validas["sg_float"] = sg_por_semilla

    activas = semillas_validas[semillas_validas["sg_float"] > 0]
    cant_activas = int(len(activas))
    pct_activas = (cant_activas / total_sv) if total_sv > 0 else 0.0

    if cant_activas > 0:
        media_sg_activas = float(activas["sg_float"].mean())
        mediana_sg_activas = float(activas["sg_float"].median())
    else:
        media_sg_activas = 0.0
        mediana_sg_activas = 0.0

    print("  ▶ Eficiencia de semillas (solo SV=1):")
    print(f"     SV_totales={total_sv} | SV_activas={cant_activas} ({pct_activas*100:.2f}%)")
    print(f"     SG_por_semilla_activa: media={media_sg_activas:.3f} | mediana={mediana_sg_activas:.3f}")


# =========================
# ANÁLISIS 6: SV=1 vs SV=0 (entropia/densidad/riesgo)
# =========================
def analizar_diferencias_sv_vs_no_sv(bloque: pd.DataFrame) -> None:
    if "es_semilla_valida" not in bloque.columns:
        print("  ▶ Comparación SV vs no-SV: (omitido, falta es_semilla_valida)")
        return

    columnas = []
    for c in ["entropia", "densidad", "riesgo"]:
        if c in bloque.columns:
            columnas.append(c)

    if len(columnas) == 0:
        print("  ▶ Comparación SV vs no-SV: (omitido, faltan entropia/densidad/riesgo)")
        return

    sv = bloque[bloque["es_semilla_valida"] == 1]
    no_sv = bloque[bloque["es_semilla_valida"] == 0]

    if len(sv) == 0 or len(no_sv) == 0:
        print("  ▶ Comparación SV vs no-SV: datos insuficientes (uno de los grupos vacío)")
        return

    print("  ▶ Diferencias SV=1 vs SV=0 (media y std):")
    for c in columnas:
        v1 = convertir_a_float_seguro(sv[c]).dropna()
        v0 = convertir_a_float_seguro(no_sv[c]).dropna()

        if len(v1) == 0 or len(v0) == 0:
            print(f"     {c:<8} (omitido: NaN tras conversión)")
            continue

        media_1 = float(v1.mean())
        std_1 = float(v1.std(ddof=1)) if len(v1) > 1 else 0.0

        media_0 = float(v0.mean())
        std_0 = float(v0.std(ddof=1)) if len(v0) > 1 else 0.0

        print(f"     {c:<8} SV=1 media={media_1:.4f} std={std_1:.4f} | SV=0 media={media_0:.4f} std={std_0:.4f}")


# =========================
# ANÁLISIS 7: FRONTERA “REAL” vs SEMILLAS MUERTAS (por criterio interno)
# =========================
def analizar_frontera_por_criterio(bloque: pd.DataFrame) -> None:
    columnas_requeridas = [
        "pasa_pureza", "pasa_densidad", "pasa_riesgo",
        "entropia", "densidad", "riesgo"
    ]
    for col in columnas_requeridas:
        if col not in bloque.columns:
            print("  ▶ Frontera por criterio: (omitido, faltan columnas)")
            return

    criterios = [
        {"nombre": "pureza",   "col_pasa": "pasa_pureza",   "col_valor": "entropia"},
        {"nombre": "densidad", "col_pasa": "pasa_densidad", "col_valor": "densidad"},
        {"nombre": "riesgo",   "col_pasa": "pasa_riesgo",   "col_valor": "riesgo"},
    ]

    print("  ▶ Análisis de frontera por criterio:")

    for crit in criterios:
        nombre = crit["nombre"]
        col_pasa = crit["col_pasa"]
        col_valor = crit["col_valor"]

        pasa = bloque[bloque[col_pasa] == 1][col_valor]
        no_pasa = bloque[bloque[col_pasa] == 0][col_valor]

        pasa_num = convertir_a_float_seguro(pasa).dropna()
        no_num = convertir_a_float_seguro(no_pasa).dropna()

        if len(pasa_num) == 0 or len(no_num) == 0:
            print(f"     {nombre:<8}: datos insuficientes (NaN tras conversión)")
            continue

        media_pasa = float(pasa_num.mean())
        std_pasa = float(pasa_num.std(ddof=1)) if len(pasa_num) > 1 else 0.0

        media_no = float(no_num.mean())
        std_no = float(no_num.std(ddof=1)) if len(no_num) > 1 else 0.0

        delta_media = media_pasa - media_no

        print(
            f"     {nombre:<8} | "
            f"pasa: n={len(pasa_num):<4} μ={media_pasa:.4f} σ={std_pasa:.4f} | "
            f"no pasa: n={len(no_num):<4} μ={media_no:.4f} σ={std_no:.4f} | "
            f"Δμ={delta_media:.4f}"
        )


# =========================
# EJECUCIÓN PRINCIPAL
# =========================
RUTA_LOGS = "."
RUTA_TEST = "."

archivos_xlsx = [
    f for f in glob.glob(os.path.join(RUTA_LOGS, "*.xlsx"))
    if not os.path.basename(f).startswith("~$")
]

dist_test = cargar_distribuciones_test(RUTA_TEST)

for ruta in archivos_xlsx:
    nombre_archivo = os.path.basename(ruta)
    dataset = (
        nombre_archivo
        .replace("log_pcsmote_x_muestra_", "")
        .replace(".xlsx", "")
    )

    df = pd.read_excel(ruta, engine="openpyxl")
    validar_columnas_obligatorias(df, nombre_archivo)

    print("\n" + "=" * 100)
    print(f"Dataset: {dataset}")
    print("=" * 100)

    for configuracion, bloque in df.groupby("configuracion"):
        print(f"\n{configuracion}")
        print("-" * len(configuracion))

        # 1) Resumen SG/SV por clase
        resumen = construir_resumen_sg_sv_por_clase(bloque)
        total_sg = imprimir_resumen_sg_sv(resumen)

        # 2) Desalineación vs TEST
        if total_sg > 0:
            imprimir_desalineacion_vs_test(dataset, resumen, dist_test)

        # 3) Entropía (dispersión) en semillas válidas
        analizar_entropia_en_semillas_validas(bloque)

        # 4) Correlación entropía vs SG_por_semilla
        analizar_correlacion_entropia_vs_sg(bloque)

        # 5) Eficiencia de semillas
        analizar_eficiencia_de_semillas(bloque)

        # 6) SV=1 vs SV=0
        analizar_diferencias_sv_vs_no_sv(bloque)

        # 7) Frontera por criterio (pasa_* vs no pasa_*)
        analizar_frontera_por_criterio(bloque)



Dataset: ecoli

PRD70_PR30_CPprop_UD060_UR040_Upp6000_I0
----------------------------------------
im     SV=0   SG=0    SG/SV=0.00
imU    SV=0   SG=0    SG/SV=0.00
om     SV=0   SG=0    SG/SV=0.00
pp     SV=0   SG=0    SG/SV=0.00

  ▶ Total sintéticos (SG): 0
  ▶ Cobertura de clases  : 0/4
  ⚠ Configuración inválida (SG = 0)
  ▶ Entropía: sin semillas válidas
  ▶ Correlación entropía↔SG: datos insuficientes (n<3)
  ▶ Eficiencia semillas: sin semillas válidas
  ▶ Comparación SV vs no-SV: datos insuficientes (uno de los grupos vacío)
  ▶ Análisis de frontera por criterio:
     pureza  : datos insuficientes (NaN tras conversión)
     densidad | pasa: n=92   μ=0.9534 σ=0.0950 | no pasa: n=54   μ=0.2725 σ=0.2310 | Δμ=0.6809
     riesgo   | pasa: n=131  μ=0.0513 σ=0.0905 | no pasa: n=15   μ=0.7238 σ=0.2191 | Δμ=-0.6726

PRD70_PR30_CPprop_UD060_UR040_Upp6000_I1
----------------------------------------
im     SV=0   SG=0    SG/SV=0.00
imU    SV=0   SG=0    SG/SV=0.00
om     SV=0   SG=0    SG/

C:\Users\User\AppData\Roaming\Python\Python311\site-packages\scipy\stats\_stats_py.py:5445: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))
C:\Users\User\AppData\Roaming\Python\Python311\site-packages\scipy\stats\_stats_py.py:5445: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))


     riesgo   SV=1 media=0.0000 std=0.0000 | SV=0 media=0.3040 std=0.3003
  ▶ Análisis de frontera por criterio:
     pureza  : datos insuficientes (NaN tras conversión)
     densidad | pasa: n=44   μ=0.8994 σ=0.1704 | no pasa: n=4    μ=0.2143 σ=0.0825 | Δμ=0.6851
     riesgo   | pasa: n=40   μ=0.1929 σ=0.2009 | no pasa: n=8    μ=0.8214 σ=0.0661 | Δμ=-0.6286

PRD85_PR45_CPprop_UD040_UR060_Upp060_I1
---------------------------------------
eccentricity SV=0   SG=0    SG/SV=0.00
missing_tooth SV=0   SG=0    SG/SV=0.00
root_crack SV=0   SG=0    SG/SV=0.00
surface_fault SV=1   SG=14   SG/SV=14.00
tooth_chipped_fault SV=0   SG=0    SG/SV=0.00

  ▶ Total sintéticos (SG): 14
  ▶ Cobertura de clases  : 1/5
  ▶ Entropía: todos NaN tras conversión
  ▶ Correlación entropía↔SG: datos insuficientes (n<3)
  ▶ Eficiencia de semillas (solo SV=1):
     SV_totales=1 | SV_activas=1 (100.00%)
     SG_por_semilla_activa: media=14.000 | mediana=14.000
  ▶ Diferencias SV=1 vs SV=0 (media y std):
     entropia

C:\Users\User\AppData\Roaming\Python\Python311\site-packages\scipy\stats\_stats_py.py:5445: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))
C:\Users\User\AppData\Roaming\Python\Python311\site-packages\scipy\stats\_stats_py.py:5445: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))
